In [1]:
import pandas as pd
import pyodbc
import os
from sql_pass import *

# Justin Do and Defne Sener 

password = sql_pass
username = 'justindo47'
default_db = "Fall_2022"
server = "localhost"    # has to be localhost not johndroescher unless using diff ide 
 
# either one must keep encrypt off and trust the server to prevent certification err
cnxn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};SERVER=%s;DATABASE=%s;UID=%s;PWD=%s;Encrypt=no;TrustServerCertificate=yes" %
    (server, default_db, username, password)
)
"""
Idea:
We follow good restaurants (now we need to find these "good" restaurants) 
    What do we need?
        - Score => Tells us their inspection grade regardless of null grades ( A (0-13) B (14-27) C (>=28) )
        - Record - Inspection Date => gives us the time they've been up
            -- (record date being "The date when the extract was run to produce this data set ")
        - Critical Flag => we CANNOT have critical violation bc => contribute to foodborne illness    

Explanation of SQL script:

WHERE dba NOT LIKE 'Test%' -->  Dba had Test test so we removed those (bc theyre testing)
WHERE score IS NOT NULL --> We need to know about their grades (less violation = better standing)

"""
sql_query = pd.read_sql("""
        SELECT dba,
               boro,
               zipcode,
               cuisine_description,
               score,
               grade,
               critical_flag,
               violation_description,
               inspection_date,
               record_date,
               datediff(year, inspection_date, record_date) as record_inspection_diff
        FROM restaurant_data as rd 
            LEFT JOIN violation_data as vd
                ON rd.camis = vd.camis
        WHERE dba NOT LIKE 'Test%' AND score IS NOT NULL
        """, con=cnxn)
# nyc_zip_codes = pd.read_csv(r'nyc-zip-codes.csv')
rest_viol_df = pd.DataFrame(sql_query)  # DataFrame Shape: (235989, 5) 235989 entries 5 columns

# Need to work with record_inspection_diff >= 5 yrs (5 yrs of bis could go up to 20 yrs --> generational = longevity)
rest_viol_df = rest_viol_df.loc[rest_viol_df['record_inspection_diff'] >= 5]

"""
We need to replace null Grades with our given score so... 
    - Instead of looping through each row and running a binary tree we should:
        - use loc to change values 
        - chain filters in the first argument using & 

Once we finish adding the grade values:
    - Let's double check that there are no null grades 
    - make a groupby with boro > grade > count
"""

# Base DataFrame
rest_viol_df.loc[(rest_viol_df['grade'].isnull()) & (rest_viol_df['score'] <= 13), 'grade'] = 'A'
rest_viol_df.loc[
    (rest_viol_df['grade'].isnull()) & (rest_viol_df['score'] > 13) & (rest_viol_df['score'] <= 27), 'grade'] = 'B'
rest_viol_df.loc[(rest_viol_df['grade'].isnull()) & (rest_viol_df['score'] >= 28), 'grade'] = 'C'
# sort by inspect date (desc high-low) most recent inspection (bc drop_dup will take the FIRST inspection it sees from dba)
rest_viol_df.sort_values(by=['inspection_date', 'score'], ascending=[0, 0], inplace=True)
print(rest_viol_df)

                               dba       boro zipcode cuisine_description  \
160690  FLAVAS INTERNATIONAL GRILL      Bronx   10451           Caribbean   
163179  FLAVAS INTERNATIONAL GRILL      Bronx   10451           Caribbean   
165809  FLAVAS INTERNATIONAL GRILL      Bronx   10451           Caribbean   
183452  FLAVAS INTERNATIONAL GRILL      Bronx   10451           Caribbean   
190475  FLAVAS INTERNATIONAL GRILL      Bronx   10451           Caribbean   
...                            ...        ...     ...                 ...   
233466                     ESCOLAS     Queens   11106      Latin American   
235275                     ESCOLAS     Queens   11106      Latin American   
144456                     ESCOLAS     Queens   11106      Latin American   
177019                     ESCOLAS     Queens   11106      Latin American   
179048      FALAFEL OFF THE CORNER  Manhattan   10128       Jewish/Kosher   

        score grade critical_flag  \
160690     44     C  Not Critical   
1

In [2]:
# no duplicate dba for cuisine counts --> keeps the first one that it sees
non_dup_rest_viol_df = rest_viol_df.drop_duplicates(subset=['dba']).loc[rest_viol_df['critical_flag'] != 'Critical']
print(non_dup_rest_viol_df)

                                       dba           boro zipcode  \
160690          FLAVAS INTERNATIONAL GRILL          Bronx   10451   
72213   HEALTHY EATERS JAMAICAN RESTAURANT         Queens   11412   
156378                                YVES      Manhattan   10013   
14938                         ROUND 2 CAFE       Brooklyn   11203   
61426              THE RIDGEWOOD ALE HOUSE         Queens   11385   
...                                    ...            ...     ...   
69174           ROMANCE BAR AND RESTAURANT  Staten Island   10303   
139334                          RMM LOUNGE          Bronx   10455   
58376                       LYCEUM THEATRE      Manhattan   10036   
53611                MANHATTAN TERRACE BAR         Queens   11420   
19778               FALAFEL OFF THE CORNER      Manhattan   10128   

       cuisine_description  score grade   critical_flag  \
160690           Caribbean     44     C    Not Critical   
72213            Caribbean      7     A    Not Critic

In [3]:
# Lower Competition with less cuisines
cuisine_groups = non_dup_rest_viol_df.groupby(['boro', 'cuisine_description']).count()
less_cuisine_groups = cuisine_groups.loc[cuisine_groups['dba'] <= 10]['dba']
print(less_cuisine_groups)

boro           cuisine_description     
Bronx          African                     2
               Asian/Asian Fusion          1
               Bakery Products/Desserts    2
               Coffee/Tea                  4
               Frozen Desserts             1
                                          ..
Staten Island  Japanese                    1
               Latin American              1
               Mexican                     2
               Pizza                       2
               Polish                      1
Name: dba, Length: 143, dtype: int64


In [4]:
# We need to find out which restaurants have these cuisines so lets loops through the index and find the restaurant they counted
less_cuisine_df = pd.DataFrame()
for boro, cuisine_type in less_cuisine_groups.index:
    # for less_cuisine_df
    df = non_dup_rest_viol_df.loc[(non_dup_rest_viol_df['boro'] == boro) & (non_dup_rest_viol_df['cuisine_description'] == cuisine_type)]
    less_cuisine_df = pd.concat([less_cuisine_df, df])    # concat is not an in place function so lets update of df

print(less_cuisine_df)

                                                    dba           boro  \
79907                            MAMA G AFRICAN KITCHEN          Bronx   
49577                        BRAVO AFRICAN RESTAURANT 2          Bronx   
55460                               LIN'S ASIAN CUISINE          Bronx   
11893                             CONTI'S PASTRY SHOPPE          Bronx   
97792   AL CHOLO WEST INDIAN BAKERY & HEALTH FOOD STORE          Bronx   
...                                                 ...            ...   
25754                                       ADOBE BLUES  Staten Island   
125347                                   EL LOBITO LOCO  Staten Island   
16492                               PRIME PIZZA KITCHEN  Staten Island   
133219                       TONY'S ORIGINAL'S PIZZERIA  Staten Island   
158605                                     TERMINAL ONE  Staten Island   

       zipcode       cuisine_description  score grade critical_flag  \
79907    10467                   African

In [5]:
# Split our less_cuisines into A grades and B&C grades
a_less_cuisine_df = less_cuisine_df.loc[(less_cuisine_df['grade'] == 'A') & (less_cuisine_df['critical_flag'] != 'Critical')]
print(a_less_cuisine_df)

                                                      dba           boro  \
79907                              MAMA G AFRICAN KITCHEN          Bronx   
49577                          BRAVO AFRICAN RESTAURANT 2          Bronx   
11893                               CONTI'S PASTRY SHOPPE          Bronx   
97792     AL CHOLO WEST INDIAN BAKERY & HEALTH FOOD STORE          Bronx   
69406   FORDHAM U - SALT & SESAME AT FACULTY MEMORIAL ...          Bronx   
...                                                   ...            ...   
25754                                         ADOBE BLUES  Staten Island   
125347                                     EL LOBITO LOCO  Staten Island   
16492                                 PRIME PIZZA KITCHEN  Staten Island   
133219                         TONY'S ORIGINAL'S PIZZERIA  Staten Island   
158605                                       TERMINAL ONE  Staten Island   

       zipcode       cuisine_description  score grade critical_flag  \
79907    10467  

In [6]:
bc_less_cuisine_df = less_cuisine_df.loc[(less_cuisine_df['grade'] == 'B') | (less_cuisine_df['grade'] == 'C')].sort_values(by='grade', ascending=1)
print(bc_less_cuisine_df)

                                      dba           boro zipcode  \
55460                 LIN'S ASIAN CUISINE          Bronx   10462   
3532        TEQUILA CHITO'S MEXICAN GRILL      Manhattan   10011   
6866           COLES DOCK SIDE RESTAURANT  Staten Island   10308   
7529                     LUNA CAFE LOUNGE      Manhattan   10003   
45625                 KISS MY SLICE PIZZA      Manhattan   10022   
113                                 SPICE      Manhattan   10003   
10766                          THAI VILLA      Manhattan   10003   
17243                             MAMALEE         Queens   11364   
75487                       ASIAN GOURMET         Queens   11355   
117013                   DHAKA RESTAURANT         Queens   11432   
41640                        MAGGIE MAE'S         Queens   11104   
20642                  CAFE MORNING OF NY         Queens   11358   
11370                 SERBIAN ASSOCIATION         Queens   11385   
106692          MARYANN'S WOODSIDE EVENTS       

In [7]:
# We need to compare A and BC grades 
a_viol_info_df = a_less_cuisine_df[['dba', 'boro', 'cuisine_description', 'grade', 'critical_flag', 'violation_description', 'inspection_date' ]]
# less_cuisine_groups = cuisine_groups.loc[cuisine_groups['dba'] <= 10]['dba']
a_viol_info_group = a_viol_info_df.groupby(['boro', 'cuisine_description']).count()
a_viol_info_group = a_viol_info_group.loc[a_viol_info_group['dba'] <= 3]['dba']
print(a_viol_info_group)

boro           cuisine_description     
Bronx          African                     2
               Bakery Products/Desserts    2
               Coffee/Tea                  2
               Frozen Desserts             1
               Greek                       1
                                          ..
Staten Island  Indian                      1
               Latin American              1
               Mexican                     2
               Pizza                       2
               Polish                      1
Name: dba, Length: 101, dtype: int64


In [8]:
bc_viol_info_df = bc_less_cuisine_df[['dba', 'boro', 'cuisine_description', 'grade', 'critical_flag', 'violation_description', 'inspection_date' ]]
bc_viol_info_group = bc_viol_info_df.groupby(['boro', 'cuisine_description']).count()
print(bc_viol_info_group)

                                   dba  grade  critical_flag  \
boro          cuisine_description                              
Bronx         Asian/Asian Fusion     1      1              1   
              Coffee/Tea             1      1              1   
              Italian                1      1              1   
              Latin American         3      3              3   
              Pizza                  1      1              1   
              Seafood                1      1              1   
Brooklyn      Asian/Asian Fusion     1      1              1   
              Chinese                2      2              2   
              Jewish/Kosher          4      4              4   
              Mexican                1      1              1   
              Middle Eastern         1      1              1   
              Other                  1      1              1   
              Seafood                1      1              1   
              Spanish                1  

In [9]:
# Combined Dataframe with cuisine descriptions that have a grade of A AND B&C  
abc_viol_info_df = pd.DataFrame()
for boro, cuisine in bc_viol_info_group.index:
    if (boro, cuisine) in a_viol_info_group.index:
        df = non_dup_rest_viol_df.loc[(non_dup_rest_viol_df['boro'] == boro) & (non_dup_rest_viol_df['cuisine_description'] == cuisine)]
        abc_viol_info_df = pd.concat([abc_viol_info_df, df])

In [10]:
# Counting the Grades in each boro/cuisine
abc_viol_info_group = abc_viol_info_df.groupby(['boro', 'cuisine_description', 'grade'])['dba'].count()

In [11]:
# Final Frame for decision making
split_a_viol_info = abc_viol_info_df.loc[abc_viol_info_df['grade'] == 'A'].sort_values(by=['boro', 'cuisine_description'], ascending=[0,0])
print(split_a_viol_info)

                                                      dba       boro zipcode  \
42309                                            RW PRIME     Queens   11420   
32486                             BOCA JUNIORS STEAKHOUSE     Queens   11373   
27485                               LUNERA MODERN MEXICAN     Queens   11103   
29846                                        CASA GALICIA     Queens   11103   
14745                                            EL TUCAN     Queens   11373   
135966                       OFF THE HOOK RAW BAR & GRILL     Queens   11103   
111436                                             ANGLER     Queens   11369   
76650                            MELODY CAFE & RESTAURANT     Queens   11103   
77666                                               DUZAN     Queens   11103   
45026                                   AIR FRANCE LOUNGE     Queens    None   
72213                  HEALTHY EATERS JAMAICAN RESTAURANT     Queens   11412   
105197              GOLDEN KRUST CARIBBE

In [13]:
split_bc_viol_info = abc_viol_info_df.loc[(abc_viol_info_df['grade'] == 'B') | (abc_viol_info_df['grade'] == 'C')].sort_values(by=['boro','score'], ascending=[0,0])
print(split_bc_viol_info)

                                      dba       boro zipcode  \
13726                      BEAN'S & LAGER     Queens   11106   
11416                                AGED     Queens   11375   
95556   GT RICEBOWL RESTAURANT AND LOUNGE     Queens   11419   
33983                        OASIS KD BAR     Queens   11368   
49891                       BICHEIROS BAR     Queens   11421   
117013                   DHAKA RESTAURANT     Queens   11432   
120272             HOT AND JUICY CRAWFISH     Queens   11354   
119566               DIRTY PIERRES BISTRO     Queens   11375   
113                                 SPICE  Manhattan   10003   
10766                          THAI VILLA  Manhattan   10003   
7529                     LUNA CAFE LOUNGE  Manhattan   10003   
21016                     LUI'S THAI FOOD  Manhattan   10003   
39518                     MAOZ VEGETARIAN  Manhattan    None   
8123                        GOOD TASTE 88   Brooklyn   11220   
6131                       GOOD FRIENDS 